Lesson 11 Chapter 5.1 APIリクエストの準備

In [3]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

Lesson 11 Chapter 5.2 関数の定義

In [4]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})


In [5]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://ekitan.com/event/station-2590',
   'title': '東京駅周辺のイベント - 駅探',
   'content': '1. 駅探 2. 東京駅の時刻表・乗り換え 3. # 東京駅周辺のイベント ## 東京駅のイベント一覧 **1〜10件**／80件（新着順） * #### 親子の安心を高める子育て講座～心理学を活かして～ 子どもとのアタッチメントを学ぶ 期間2025年10月20日(月) 会場甲南大学ネットワークキャンパス東京 + 東京都 千代田区 + 東京駅／大手町駅(東京)／三越前駅 1. 無料 2. 屋内 * #### 京都アカデミアフォーラムin丸の内共催「京都新聞講座in東京『京都あれこれ』」 新たな「鬼平犯科帳」ができるまで 期間2025年9月8日(月) 会場京都大学東京オフィス\u3000会議室AB + 東京都 千代田区 + 東京駅／二重橋前駅／大手町駅(東京)駅 1. 無料 2. 屋内 * #### 丸の内ストリートマーケット by Creema 人気のクラフトイベントが6年ぶりに復活！ 期間2025年8月29日(金)～8月30日(土) 会場行幸地下通路 + 東京都 千代田区 + 二重橋前駅／東京駅／大手町駅(東京)駅 期間2025年8月1日(金)～8月24日(日) + 東京駅／二重橋前駅／大手町駅(東京)駅 期間2025年11月16日(日) + 大手町駅(東京)／竹橋駅／東京駅 期間2025年11月1日(土) + 二重橋前駅／東京駅／大手町駅(東京)駅 期間2025年10月4日(土)～10月5日(日) + 二重橋前駅／東京駅／大手町駅(東京)駅 期間2025年8月13日(水) + 東京駅／日本橋駅(東京)／大手町駅(東京)駅 期間2025年8月28日(木) + 東京駅／二重橋前駅／大手町駅(東京)駅 期間2025年8月7日(木) + 東京駅／二重橋前駅／大手町駅(東京)駅 * 京橋(東京) * 大手町(東京) * 日本橋(東京) * 東京メトロ日比谷線 八丁堀駅 徒歩2分 1K / 26.78m2 / 5階 東京メトロ日比谷線 八丁堀駅 徒歩3分 1K / 33.58m2 / 4階 / 築10年 東京メトロ有楽町線 銀座一丁目駅 徒歩2分 ワンルーム / 

Lesson 11 Chapter 5.3 ツールの定義

In [6]:
# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

Lesson 11 Chapter 5.4 言語モデルへの質問を行う関数

In [7]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

Lesson 11 Chapter 5.5 ツール呼び出しが必要な場合の処理を行う関数

In [8]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

Lesson 11 Chapter 5.6 ユーザーからの質問を処理する関数

In [9]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

Lesson 11 Chapter 5.7 動作確認

In [10]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

- 東京都の面積は約2,194平方キロメートルです。
- 沖縄県の面積は約2,271平方キロメートルです。

したがって、沖縄県は東京都よりもわずかに広いです。


In [11]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
最近1ヶ月以内に東京駅周辺で開催されるイベントについての情報をいくつかご紹介します。

1. **東京駅（東京府）周辺のイベント情報**
   - **イベント名**: 東京駅周辺のイベント
   - **日時**: 2025年に複数のイベントが予定されています。
   - **内容**: 詳細は、[WalkerPlus](https://www.walkerplus.com/event_list/ar0313/sc309880d/) で確認できます。

2. **英文展示「今日行ける！東京駅周辺のあすゆうイベント」**
   - **日時**: 2025年8月19日から8月25日まで
   - **場所**: 東京駅周辺
   - **内容**: 国際アートや地元の魅力を発信するイベント。詳細は、[Enjoy Tokyo](https://www.enjoytokyo.jp/event/list/sta200101/its04/) を参照してください。

3. **GRANSTAの新着情報**
   - **内容**: 東京駅構内にあるGRANSTAでの新しいショップなどの情報が公開されています。詳細は[GRANSTA](https://www.gransta.jp/news/)のサイトをチェックしてください。

4. **CREATIVE MUSEUM TOKYO**
   - **開催期間**: 2025年9月12日から9月14日
   - **内容**: クリエイティブな展覧会。詳細は[Enjoy Tokyo](https://www.enjoytokyo.jp/event/list/area1306/)を確認してください。

これらの情報は、東京駅周辺の最近のイベントの一部です。興味があるイベントがあれば、公式サイトで詳細を確認することをおすすめします。


Lesson 11 Chapter 6.1 ソースコードで確認

In [14]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

role = input("チャットボットのキャラクターを設定してください:")
messages.append({"role":"system", "content": role.strip()})

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        for i, msg in enumerate(messages):
            if msg["role"] == "user":
                del_message = messages.pop(i)
                if len(messages) <= 8:
                    break

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:こんにちは！'

こんにちは！今日はどんなことをお手伝いできますか？


'質問:東北６県は？'

東北地方は、日本の北東部に位置する地域で、以下の6つの県から構成されています。

1. 青森県（あおもりけん）
2. 岩手県（いわてけん）
3. 宮城県（みやぎけん）
4. 秋田県（あきたけん）
5. 山形県（やまがたけん）
6. 福島県（ふくしまけん）

これらの県は、自然の美しさや豊かな文化、歴史的な名所があることでも知られています。


'質問:宮城県のお土産について検索した結果を教えて'

宮城県のお土産に関する情報をいくつか挙げてみます。

1. **宮城のお土産ランキング**: 宮城県で人気のお土産をランク付けした情報がいくつか提供されています。地元で作られたお菓子や特産品が含まれています。たとえば「ずんだ餅」や「萩の月」などが有名です。 [詳細はこちら](https://www.jalan.net/omiyage/040000/)。

2. **お土産の種類**: 宮城県の特産品としては、牛たんや、萩の月、ずんだ餅、仙台味噌、海産物などが挙げられます。特にずんだ関連は観光客に好評です。 [さらに詳しく](https://tsplus.asahi.co.jp/articles/gift/75865/)。

3. **おすすめのお土産**: 具体的な商品名として「萩の月」、「ずんだ餅」、「鬼がらし」、「ぶっかけ冷やしそば」などが紹介されています。これらは商品の質や味を考慮して選ばれています。 [詳しくはここをチェック](https://tohokuru.jp/blogs/feature/miyage_miyagi?srsltid=AfmBOorVPNK74a4oU_pPB68PyQTJWVzyuS6BP_DWQehTK6nSukbYXHMz)。

4. **お土産店や購入方法**: 宮城県内の店舗や観光地でこれらの特産品を購入することができます。観光スポットの周辺には、特産物を扱っている店舗も多くあります。 [ショッピング情報まとめ](https://sakidori.co/article/1320827)。

これらの情報を参考に、宮城県を訪れた際にはぜひ地元の特産品をお土産として楽しんでみてください！


'質問:青森県のお土産について検索した結果を教えて'

青森県のお土産には多くの魅力的な選択肢があります。以下はその一部です。

1. **青森のリンゴスナック**:
   - 青森県産のリンゴを使用した健康的なおやつで、フリーズドライにし、添加物を含まずに自然の風味と甘さを閉じ込めています。

2. **ねぷたせんべい**:
   - 青森県黒石市の老舗菓子店「柴川つがるせんべい」で作られた、つがる米を使用した米菓です。

3. **青森の伝統的な工芸品や加工食品**:
   - 地元の郷土料理や新鮮な魚介類、肉、野菜などが豊富にあります。地域の特産品を使った商品が多く、地元のお土産店や市場で購入できます。

4. **青森の鮮魚や海産物**:
   - 新鮮な魚介類は、地元で太陽の光を浴びて育った自然豊かな環境で採れたものです。

これらのお土産は、青森の風味を楽しむだけでなく、県の文化や伝統を感じることもできるすばらしい品々です。興味のある方は、ぜひ青森を訪れてみてください。

詳細な情報は、次のリンクからご覧いただけます:
- [青森のおすすめお土産](https://monita.online/article/844)
- [青森の特産品について](https://aomori-tourism.com/gourmet/index.html)

---ご利用ありがとうございました！---
